# **Caso 1 — Telecomunicaciones: identificar operadores ineficaces**

**Objetivo**
Identificar operadores ineficaces considerando:
(1) Muchas llamadas entrantes perdidas, (2) tiempos de espera altos en llamadas entrantes y (3) pocas llamadas salientes si aplica.

**KPIs por (user_id, operator_id) :**
- missed_rate_in = missed_in / calls_in
- avg_wait_in = (total_call_duration_in - call_duration_in) / calls_in
- out_calls_per_day = calls_out / días_activos
- ineff_score = mean( z(missed_rate_in), z(avg_wait_in), -z(out_calls_per_day) )

**Datos**
telecom_dataset_new.csv — actividad de llamadas
telecom_clients.csv — plan tarifario del cliente

In [ ]:
# %% C1_00_configuracion

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os  # para crear carpeta de salida
import math

pd.set_option("display.max_columns", 120)

# Rutas directas (alineado a sprints)
calls_path   = "./datasets/telecom_dataset_new.csv"
clients_path = "./datasets/telecom_clients.csv"

# Carpeta de resultados (necesaria para guardar CSVs)
os.makedirs("./outputs/processed", exist_ok=True)

# **Carga de datos**

In [ ]:
# %% C1_01_carga

df_calls_raw   = pd.read_csv(calls_path)
df_clients_raw = pd.read_csv(clients_path)

print("Archivos cargados")
print("Llamadas:", df_calls_raw.shape, "| Clientes:", df_clients_raw.shape)


# **2) Exploración — Llamadas (df_calls_raw)**

In [ ]:
# %% C1_02_exploracion_llamadas

# Vista general
df_calls_raw.info()

# Muestra
print("\nHEAD (5 filas):")
print(df_calls_raw.head(5))

# Estadísticas
print("\nDESCRIBE (incluye todo tipo de columnas):")
print(df_calls_raw.describe(include='all'))


# **3) Exploración — Clientes (df_clients_raw)**

In [ ]:
# %% C1_03_exploracion_clientes

# Vista general
df_clients_raw.info()

# Muestra
print("\nHEAD (5 filas):")
print(df_clients_raw.head(5))

# Estadísticas
print("\nDESCRIBE (incluye todo tipo de columnas):")
print(df_clients_raw.describe(include='all'))

# **4) Contar y eliminar duplicados (si los hay)**

In [ ]:
# %% C1_04_duplicados

dup_calls = int(df_calls_raw.duplicated().sum())
dup_clients = int(df_clients_raw.duplicated().sum())
print(f"Duplicados en llamadas: {dup_calls} | Duplicados en clientes: {dup_clients}")

# Solo eliminar si existen
df_calls = df_calls_raw.drop_duplicates().reset_index(drop=True) if dup_calls > 0 else df_calls_raw
df_clients = df_clients_raw.drop_duplicates().reset_index(drop=True) if dup_clients > 0 else df_clients_raw

if dup_calls > 0:
    print(f"Eliminados {dup_calls} duplicados en llamadas.")
if dup_clients > 0:
    print(f"Eliminados {dup_clients} duplicados en clientes.")

# **5) Conversión de tipos y variables derivadas (solo si existen)**

In [ ]:
# %% C1_05_tipos_y_derivadas

# Fechas a datetime si están presentes
if 'date' in df_calls.columns:
    df_calls['date'] = pd.to_datetime(df_calls['date'], errors='coerce')
if 'date_start' in df_clients.columns:
    df_clients['date_start'] = pd.to_datetime(df_clients['date_start'], errors='coerce')

# Variable de espera: total - conversación (cap en 0 por seguridad simple)
if {'total_call_duration','call_duration'}.issubset(df_calls.columns):
    df_calls['wait_time'] = (df_calls['total_call_duration'] - df_calls['call_duration']).clip(lower=0)

# Log mínimo
if 'date' in df_calls.columns:
    print("Rango de fechas (llamadas):", df_calls['date'].min(), "a", df_calls['date'].max())
print("Tamaño actual — llamadas:", df_calls.shape, "| clientes:", df_clients.shape)

# **6) KPIs por operador (excluir filas sin operator_id solo para la agregación)**

In [ ]:
# %% C1_06_kpis

if 'operator_id' not in df_calls.columns:
    raise ValueError("No existe la columna 'operator_id' en el dataset de llamadas.")

# Usar solo filas con operador para KPIs por operador
df_calls_op = df_calls[df_calls['operator_id'].notna()].copy()

# Agregar por cliente–operador–dirección
df_agg = df_calls_op.groupby(['user_id','operator_id','direction'], as_index=False).agg(
    calls=('calls_count','sum'),
    missed=('is_missed_call','sum'),
    call_duration=('call_duration','sum'),
    total_call_duration=('total_call_duration','sum'),
    wait_time=('wait_time','sum'),
)

# Pivot a columnas *_in y *_out
df_pivot = (df_agg
            .pivot_table(index=['user_id','operator_id'], columns='direction',
                         values=['calls','missed','call_duration','total_call_duration','wait_time'],
                         aggfunc='sum')
            .fillna(0))

# Aplanar columnas
df_pivot.columns = ['_'.join(c) for c in df_pivot.columns.to_flat_index()]
df_pivot = df_pivot.reset_index()

# Accesos seguros a columnas (si no existen, crear serie de ceros del mismo largo)
zero = lambda: pd.Series(0, index=df_pivot.index)
calls_in  = df_pivot.get('calls_in', zero())
missed_in = df_pivot.get('missed_in', zero())
talk_in   = df_pivot.get('call_duration_in', zero())
total_in  = df_pivot.get('total_call_duration_in', zero())

# KPIs entrantes
df_pivot['missed_rate_in'] = np.where(calls_in > 0, missed_in / calls_in, np.nan)
df_pivot['avg_wait_in']    = np.where(calls_in > 0, (total_in - talk_in) / calls_in, np.nan)

# Salientes por día observados (si hay fechas)
if 'date' in df_calls_op.columns:
    df_span = df_calls_op.groupby(['user_id','operator_id'])['date'].agg(['min','max']).reset_index()
    df_span['days'] = (df_span['max'] - df_span['min']).dt.days.clip(lower=1)
    df_pivot = df_pivot.merge(df_span[['user_id','operator_id','days']],
                              on=['user_id','operator_id'], how='left')
    calls_out = df_pivot.get('calls_out', zero())
    df_pivot['out_calls_per_day'] = np.where(df_pivot['days'] > 0, calls_out / df_pivot['days'], 0.0)
else:
    df_pivot['days'] = np.nan
    df_pivot['out_calls_per_day'] = np.nan

# Enriquecer con plan del cliente (si está)
if {'user_id','tariff_plan'}.issubset(df_clients.columns):
    df_pivot = df_pivot.merge(df_clients[['user_id','tariff_plan']], on='user_id', how='left')

print("KPIs construidos (muestra):")
print(df_pivot.head(10))


# **7) Score de ineficiencia y ranking**

In [ ]:
# %% C1_07_score_ranking

# z-score simples como en los sprints
z_missed = (df_pivot['missed_rate_in'] - df_pivot['missed_rate_in'].mean(skipna=True)) / df_pivot['missed_rate_in'].std(skipna=True)
z_wait   = (df_pivot['avg_wait_in']    - df_pivot['avg_wait_in'].mean(skipna=True))    / df_pivot['avg_wait_in'].std(skipna=True)
z_out    = (df_pivot['out_calls_per_day'] - df_pivot['out_calls_per_day'].mean(skipna=True)) / df_pivot['out_calls_per_day'].std(skipna=True)

df_pivot['ineff_score'] = pd.concat([z_missed, z_wait, -z_out], axis=1).mean(axis=1)

df_top10 = df_pivot.sort_values('ineff_score', ascending=False).head(10)

# Exportes (solo lo necesario)
df_pivot.to_csv("./outputs/processed/telecom_operator_kpis.csv", index=False)
df_top10.to_csv("./outputs/processed/telecom_top10_inefficient_operators.csv", index=False)

print("Top 10 operadores ineficaces (muestra):")
print(df_top10[['user_id','operator_id','tariff_plan','missed_rate_in','avg_wait_in','out_calls_per_day','ineff_score']])

# **8) EDA visual (para dashboard)**

In [ ]:
# %% C1_08_graficos

# 1) Histograma de duración de llamada
plt.figure()
df_calls['call_duration'].dropna().plot(kind='hist', bins=30)
plt.title('Distribución de duración de llamadas')
plt.xlabel('Segundos'); plt.ylabel('Frecuencia')
plt.tight_layout(); plt.show()

# 2) Pie: internas vs externas (rotulando nulos como 'desconocido')
plt.figure()
if 'internal' in df_calls.columns:
    labels = df_calls['internal'].fillna('desconocido').astype(str)
    share = labels.value_counts(dropna=False)
    plt.pie(share.values, labels=share.index.astype(str))
else:
    plt.pie([1], labels=['sin columna internal'])
plt.title('Participación: llamadas internas vs externas')
plt.tight_layout(); plt.show()

# 3) Histograma: llamadas por día
if 'date' in df_calls.columns and 'calls_count' in df_calls.columns:
    daily_calls = df_calls.groupby('date')['calls_count'].sum().reset_index()
    plt.figure()
    daily_calls['calls_count'].plot(kind='hist', bins=30)
    plt.title('Distribución de llamadas por día')
    plt.xlabel('Llamadas'); plt.ylabel('Frecuencia de días')
    plt.tight_layout(); plt.show()


# **9) Prueba de hipótesis**

**Hipótesis a contrastar**
- **H₀:** la proporción de llamadas entrantes perdidas es igual entre operadores “ineficientes” (Top 25% del ineff_score) y el resto.
- **H₁:** la proporción de llamadas entrantes perdidas es mayor en el grupo “ineficiente”.

**Métrica:** proporción = missed_in / calls_in.
**Test:** z-test para diferencia de proporciones (aprox. normal, una cola).
**Criterio de decisión:** α = 0,05. Rechazamos H₀ si p (una cola) < 0,05.


In [ ]:
# %% C1_09_prueba_hipotesis

# 1) Definir grupo "ineficiente" como Top 25% del ineff_score
q75 = df_pivot['ineff_score'].quantile(0.75)
df_pivot['inefficient_flag'] = (df_pivot['ineff_score'] >= q75)

# 2) Conteos por grupo (entrantes)
calls_in_series  = df_pivot.get('calls_in', pd.Series(0, index=df_pivot.index))
missed_in_series = df_pivot.get('missed_in', pd.Series(0, index=df_pivot.index))

grp = df_pivot.groupby('inefficient_flag', as_index=False).agg(
    missed_in=('missed_in','sum'),
    calls_in =('calls_in','sum')
)

if grp.shape[0] < 2 or (grp['calls_in'] == 0).any():
    print("No es posible ejecutar el z-test: falta algún grupo o hay 0 llamadas entrantes en un grupo.")
else:
    g1 = grp.loc[grp['inefficient_flag']==True]
    g2 = grp.loc[grp['inefficient_flag']==False]
    n1 = int(g1['calls_in'].values[0]); x1 = int(g1['missed_in'].values[0])
    n2 = int(g2['calls_in'].values[0]); x2 = int(g2['missed_in'].values[0])

    p1 = x1 / n1
    p2 = x2 / n2
    p_pool = (x1 + x2) / (n1 + n2)
    se = math.sqrt(p_pool * (1 - p_pool) * (1/n1 + 1/n2))
    z = (p1 - p2) / se if se > 0 else float('inf')

    Phi = lambda t: 0.5 * (1.0 + math.erf(t / math.sqrt(2.0)))
    p_value_one_sided = 1 - Phi(z)          # H1: p1 > p2
    p_value_two_sided = 2 * min(Phi(z), 1 - Phi(z))

    print("==== Z-TEST proporciones (ineficientes vs resto) ====")
    print(f"n1={n1}, x1={x1}, p1={p1:.4f}  |  n2={n2}, x2={x2}, p2={p2:.4f}")
    print(f"z={z:.3f},  p (una cola, H1: p1>p2) = {p_value_one_sided:.4g}")
    print(f"p (dos colas) = {p_value_two_sided:.4g}")

    alpha = 0.05
    if p_value_one_sided < alpha:
        print("Decisión: RECHAZAMOS H0 → mayor proporción de perdidas en el grupo ineficiente.")
    else:
        print("Decisión: NO rechazamos H0 → no hay evidencia suficiente de mayor proporción en el grupo ineficiente.")


# **10) Conclusiones**

**Del EDA y preparación mínima**

- Se verificó estructura con info(), head() y describe().
- Se eliminaron duplicados solo si existían.
- Se convirtieron date/date_start a datetime si estaban presentes.
- Se derivó wait_time = total_call_duration - call_duration con cap en 0.

**KPIs y ranking**

- missed_rate_in y avg_wait_in miden abandono y espera en entrantes.
- out_calls_per_day captura productividad outbound normalizada.
- ineff_score (promedio de z-scores) permite ordenar operadores y priorizar el Top 10.

**Prueba de hipótesis**

- Hipótesis embebida: comparar proporción de perdidas entre Top 25% ineficientes y resto.
- Conclusión depende del p-valor (una cola, α=0,05) impreso por el z-test:
    - Si p < 0,05 → evidencia de mayor proporción de perdidas en el grupo ineficiente.
    - Si p ≥ 0,05 → no hay evidencia suficiente.

**Recomendaciones**

- Alta missed_rate_in: revisar dotación, horarios pico y reglas de enrutamiento.
- Alta avg_wait_in: ajustar SLA de cola y procesos de atención inicial.
- Baja out_calls_per_day (si aplica outbound): metas, asignación de campañas y disponibilidad.
- Revisar por tariff_plan diferencias operativas entre clientes.

**Limitaciones y siguientes pasos**

- Filas sin operator_id no se usan en el ranking (no atribuibles).
- Posible extensión: hipótesis sobre tiempos de espera (p. ej., test no paramétrico) y z-scores por cliente (user_id) para comparaciones internas.

# **11) Fuentes y referencias**

1. **Pandas** – API Reference (read_csv, info, describe, head, groupby, pivot_table, isna, duplicated)
https://pandas.pydata.org/docs/reference/index.html
Carga de CSV, EDA, agregaciones y pivoteos.

2. **Pandas** – User Guide: GroupBy
https://pandas.pydata.org/docs/user_guide/groupby.html
Agrupar por user_id/operator_id/direction y sumar métricas.

3. **Pandas** – User Guide: Reshaping and Pivot Tables
https://pandas.pydata.org/docs/user_guide/reshaping.html#pivot-tables
Construcción de df_pivot con columnas _in/_out.

4. **NumPy** – Reference (estadística básica)
https://numpy.org/doc/stable/reference/
Cálculo de z-scores para componer ineff_score.

5. **Matplotlib** – Pyplot
https://matplotlib.org/stable/users/explain/pyplot/index.html
Histogramas y pie chart requeridos en el dashboard.

6. **Python os** — File and Directory Access
https://docs.python.org/3/library/os.html
Crear carpeta de salida con os.makedirs.

7. **Material del Bootcamp (Sprints 3–5, 10–12)**
(documentos del curso)
Orden EDA → limpieza mínima → KPIs → comunicación de resultados.